In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import networkx as nx

from torch.utils.data import Dataset, Subset
from torch_geometric.data import Data, DataLoader
from torch import nn

from sklearn.metrics import r2_score
from sklearn.model_selection import GroupShuffleSplit

import ray, optuna
from ray import tune
from ray.tune.search.optuna import OptunaSearch


/home/users/qiaok/venvs/graph/lib/python3.12/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /home/users/qiaok/venvs/graph/lib/python3.12/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/users/qiaok/venvs/graph/lib/python3.12/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /lib64/libc.so.6: version `GLIBC_2.32' not found (required by /home/users/qiaok/venvs/graph/lib/python3.12/site-packages/torch_cluster/_version_cuda.so)
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/home/users/qiaok/venvs/graph/lib/python3.12/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /lib64/libc.so.6: vers

In [2]:
from MetaNet import MetaNet, BioDegDataset

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
torch.cuda.device_count()


2

In [5]:
data = BioDegDataset('df_cleaned.csv','.')
#data = BioDegDatasetCached('biodataset_processed.pt')

In [6]:
for i, d in enumerate(data):
    if not hasattr(d, "compID"):
        print(i, hasattr(d, "compID"), getattr(d, "compID", None))
    elif d.compID is None:
        print(i, hasattr(d, "compID"), getattr(d, "compID", None))


In [13]:
"""
groups = np.array([d.compID for d in data])          # 1 tag per sample

outer = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
train_val_idx, test_idx = next(
    outer.split(X=np.arange(len(data)), groups=groups)   # groups in 3rd slot
)

inner = GroupShuffleSplit(n_splits=1, test_size=0.125, random_state=42)
train_idx, val_idx = next(
    inner.split(X=train_val_idx, groups=groups[train_val_idx])  # split only inside train+val
)

train_set = Subset(data, train_idx)     # 70 % of total
val_set   = Subset(data, val_idx)       # 10 %
test_set  = Subset(data, test_idx)      # 20 %

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_set,   batch_size=64, shuffle=False)
test_loader  = DataLoader(test_set,  batch_size=64, shuffle=False)
"""

/home/users/qiaok/venvs/graph/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [13]:
"""


from sklearn.preprocessing import StandardScaler
# fit only on training rows
u_train = torch.cat([data[i].u for i in train_idx], dim=0).numpy()  # [n_train, 4]
scaler_u = StandardScaler().fit(u_train)

# transform every split
for i in np.concatenate([train_idx, val_idx, test_idx]):
    data[i].u = torch.tensor(
        scaler_u.transform(data[i].u), dtype=torch.float   # keep [1,4] shape
    )
"""

In [13]:
#torch.save((data.data, data.slices),  'biodataset_processed.pt')

/Users/qiaokang-v2/miniforge3/envs/graph/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The data of the dataset is already cached, so any modifications to `data` will not be reflected when accessing its elements. Clearing the cache now by removing all elements in `dataset._data_list`. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [17]:
device = torch.device('cuda:0')

In [27]:
model = MetaNet(n_layers=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
loss_fn = nn.SmoothL1Loss()

In [28]:
def run(loader, train=False):
    ys, preds = [], []
    model.train(train)
    for batch in loader:
        batch = batch.to(device)
        pred = model(batch)
        if train:
            loss = loss_fn(pred, batch.y.squeeze(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        ys.append(batch.y.cpu())
        preds.append(pred.detach().cpu())
    ys    = torch.cat(ys).numpy()
    preds = torch.cat(preds).numpy()
    return r2_score(ys, preds)


In [30]:

for epoch in range(1000):                       # ← ONE round as requested
    r2_train = run(train_loader, train=True)
    r2_val   = run(val_loader)
    r2_test  = run(test_loader)
    print(f"epoch 1 | R²  train: {r2_train:.3f} | val: {r2_val:.3f} | test: {r2_test:.3f}")

epoch 1 | R²  train: 0.010 | val: 0.102 | test: -0.202
epoch 1 | R²  train: 0.148 | val: 0.053 | test: -0.297
epoch 1 | R²  train: 0.083 | val: 0.130 | test: -0.243
epoch 1 | R²  train: 0.138 | val: 0.137 | test: -0.075
epoch 1 | R²  train: 0.150 | val: 0.162 | test: -0.121
epoch 1 | R²  train: 0.152 | val: 0.197 | test: -0.181
epoch 1 | R²  train: 0.153 | val: 0.251 | test: 0.029
epoch 1 | R²  train: 0.268 | val: 0.254 | test: 0.059
epoch 1 | R²  train: 0.274 | val: 0.311 | test: 0.099
epoch 1 | R²  train: 0.241 | val: 0.268 | test: -0.072
epoch 1 | R²  train: 0.315 | val: 0.310 | test: -0.021
epoch 1 | R²  train: 0.275 | val: 0.273 | test: -0.266
epoch 1 | R²  train: 0.296 | val: 0.265 | test: 0.186
epoch 1 | R²  train: 0.317 | val: 0.342 | test: -0.202
epoch 1 | R²  train: 0.318 | val: -0.030 | test: -0.595
epoch 1 | R²  train: 0.213 | val: 0.361 | test: -0.122
epoch 1 | R²  train: 0.269 | val: 0.272 | test: -0.221
epoch 1 | R²  train: 0.322 | val: 0.371 | test: 0.082
epoch 1 | R²  